# Kmeans for PoS

## 0. Data Loading : unique NOUNS, VERBS, ADJ

In [4]:
import pandas as pd

# WE Loading
all_n_we = pd.read_csv('../Data/FlauBERT_WE/all_nouns_we.csv', index_col=0).drop(columns=["number", "gender"])
all_a_we = pd.read_csv('../Data/FlauBERT_WE/all_adjectives_we.csv', index_col=0).drop(columns = ["number", "gender"])
all_v_we = pd.read_csv('../Data/FlauBERT_WE/all_verb_we.csv', index_col=0)


In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

## 1. NOUN vs not-NOUN

In [17]:
import numpy as np

# label 1: NOUN, 0: not NOUN
all_n_we["noun"] = 1
all_av_we = pd.concat([all_a_we, all_v_we, all_n_we])
all_av_we["noun"] = 0

# Normalization and concatenation
df = pd.concat([all_av_we, all_n_we])
norm_df = (df - df.min()) / (df.max() - df.min())

# Removing words having multiple POS
word, count = np.unique(norm_df.index, return_counts=True)
unique_words = [x[0] for x in list(filter(lambda x: x[1] == 1, zip(word, count)))]
un_nouns =  norm_df[norm_df.index.isin(unique_words)]

In [18]:
un_nouns["noun"].value_counts()

0.0    5483
Name: noun, dtype: int64

### 1 dim

In [11]:
dim_list_1= [str(i) for i in range(512)] # all dimensions
df_clustering_1 = pd.DataFrame(columns = ["ARI"])
ari = []

for dim in dim_list_1:

    # Clustering using Kmeans
    km = KMeans(n_clusters=2, random_state=42, n_init=10)
    km.fit(norm_df[dim].values.reshape(-1, 1))

    # Compute the Adjusted Rand Index: the closer to 1, the better
    ari.append(adjusted_rand_score(norm_df["noun"], km.labels_))

df_clustering_1["ARI"] = ari
df_clustering_1.index = dim_list_1

In [12]:
top10_1 = df_clustering_1.sort_values(by = "ARI", ascending = False)
dim_top10_1 = list(top10_1.head(10).index)
top10_1.head(10)

,ARI
0,0.0
1,0.0
350,0.0
349,0.0
348,0.0
347,0.0
346,0.0
345,0.0
344,0.0
343,0.0
